## Подход 1: градиентный бустинг "в лоб"

In [1]:
import numpy as np
import pandas as pd

features = pd.read_csv('../features.csv', index_col='match_id')
features_test = pd.read_csv('../features_test.csv', index_col='match_id')

print('features', list(features), '\n')
print('features_test', list(features_test))

features.describe()

features ['start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero', 'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths', 'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero', 'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths', 'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero', 'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths', 'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills', 'd5_deaths', 'd5_items', 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_bl

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
count,9.723000e+04,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,...,97230.000000,97230.000000,97230.000000,95404.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000,97230.000000
mean,1.444232e+09,2.630999,51.517104,3.442672,1233.405801,1147.899702,11.231996,0.357009,0.362285,8.271315,...,3.349553,2.448339,0.689119,-6.901922,2332.247886,0.518503,1309.227790,1286.310820,40.599095,41.337036
std,5.515393e+06,2.835761,32.564211,1.111741,566.588895,464.111662,9.041620,0.663889,0.626704,2.497575,...,1.155609,0.813459,0.710122,40.701397,715.806850,0.499660,853.921365,851.009148,27.871645,27.064873
min,1.430199e+09,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-84.000000,900.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.440815e+09,1.000000,22.000000,3.000000,767.000000,746.000000,2.000000,0.000000,0.000000,7.000000,...,3.000000,2.000000,0.000000,-31.000000,1818.000000,0.000000,36.000000,256.000000,3.000000,3.000000
50%,1.446338e+09,1.000000,50.000000,3.000000,1175.000000,1113.000000,11.000000,0.000000,0.000000,8.000000,...,3.000000,2.000000,1.000000,-16.000000,2268.000000,1.000000,1824.000000,1798.000000,63.000000,60.000000
75%,1.448829e+09,7.000000,75.000000,4.000000,1704.000000,1479.000000,19.000000,1.000000,1.000000,10.000000,...,4.000000,3.000000,1.000000,8.000000,2778.000000,1.000000,1974.000000,1974.000000,63.000000,63.000000
max,1.450313e+09,7.000000,112.000000,6.000000,3319.000000,4332.000000,47.000000,8.000000,5.000000,34.000000,...,9.000000,9.000000,13.000000,300.000000,8452.000000,1.000000,2047.000000,2047.000000,63.000000,63.000000


Понятно, что duration, radiant_win, tower_status_radiant, tower_status_dire, barracks_status_radiant, barracks_status_dire – лишние для наших целей признаки. Убираем. 

Вообще, неочевидно, зачем нужны признаки start_time и lobby_type. По-моему, от этих признаков победа той или иной команды вообще никак не зависит.

In [2]:
X_train = features.drop(['start_time',
                         'lobby_type',
                         'duration',
                         'radiant_win',
                         'tower_status_radiant',
                         'tower_status_dire',
                         'barracks_status_radiant',
                         'barracks_status_dire'], axis=1)
Y_train = features['radiant_win']
X_test = features_test.drop(['start_time', 'lobby_type'], axis=1)

Проверим равенство кол-ва полей в train и test

In [3]:
len(list(X_train)) == len(list(X_test))

True

Проверим данные на наличие пропусков

Сначала, выведем признаки, где пропуски есть

In [4]:
nan_list = []
for e in list(X_train):
    flag = X_train[e].isnull().values.any()
    if flag:
        nan_list.append(e)
        print(e)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Теперь, смотрим количество пропусков в признаках, где они есть, в сравнении с признаками, где пропусков нет (97230)

Я эту вашу Доту не знаю, но полагаю, что в первые пять минут игры игрокам чаще всего происходят события, описываемые признаками courier и first_ward. Соответственно, наименьшее кол-во пропусков в данных есть именно в этих признаках, поскольку эти пропуски, очевидно, означают отсутствие события, описываемого признаком в первые пять минут игры. Сourier, наверное, какой-то отправленный курьер, first_ward – первое отражение атаки. Далее идёт убывание вероятности события в первые пять минут вплоть до признака first_blood_player2, каковой, наверное, есть игрок, причинивший ущерб first_blood_player1, поскольку этот самый first_blood_player может быть не героем, а ботом, миной или ещё чем.

In [5]:
X_train[nan_list].count()

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Меняем пропуски на 0.0

In [6]:
X_train.fillna(0, inplace=True, axis=1)
X_test.fillna(0, inplace=True, axis=1)

проверка

In [7]:
for e in list(X_train):
    flag = X_train[e].isnull().values.any()
    if flag:
        print(e)

Столбец с целевой переменной – radiant_win. Данные из него сейчас находятся в Y_train.

обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [9]:
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.model_selection import KFold, cross_val_score

predictions, mean_scores = {}, {}

for trees in range(10, 101, 10):    # n_estimators
    start_time = datetime.datetime.now()
    clf = gbc(n_estimators=trees)
    clf.fit(X_train, Y_train)
    print('Classification time elapsed:', datetime.datetime.now() - start_time)

    predictions[trees] = clf.predict_proba(X_test)[:, 1]
    
    folder = KFold(n_splits=5, shuffle=True)    # генератор разбиений
    start_time = datetime.datetime.now()
    mean_scores[trees] = np.mean(cross_val_score(clf, X_train, Y_train, scoring='roc_auc', cv=folder))
    print('Cross-validation time elapsed:', datetime.datetime.now() - start_time)
    print(f'{trees}: {mean_scores[trees]}')
predictions[trees]

/home/mk/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Classification time elapsed: 0:00:07.187410
Cross-validation time elapsed: 0:00:25.234638
10: 0.6649023480596046
Classification time elapsed: 0:00:12.986533
Cross-validation time elapsed: 0:00:46.560288
20: 0.6819563092996052
Classification time elapsed: 0:00:18.088130
Cross-validation time elapsed: 0:01:08.955978
30: 0.6892847426135573
Classification time elapsed: 0:00:24.017527
Cross-validation time elapsed: 0:01:32.017844
40: 0.6941703333969077
Classification time elapsed: 0:00:29.796666
Cross-validation time elapsed: 0:01:57.616060
50: 0.6976238683505217
Classification time elapsed: 0:00:37.257479
Cross-validation time elapsed: 0:02:13.063760
60: 0.7003628955998039
Classification time elapsed: 0:00:42.578459
Cross-validation time elapsed: 0:02:36.132581
70: 0.7019657241934635
Classification time elapsed: 0:00:49.225233
Cross-validation time elapsed: 0:02:50.391700
80: 0.7036504392313014
Classification time elapsed: 0:00:53.472640
Cross-validation time elapsed: 0:03:22.272168
90: 0.

array([0.48631063, 0.6660456 , 0.4380853 , ..., 0.23806388, 0.33280151,
       0.5213588 ])

## Отчёт по первому этапу:

   #### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
    first_blood_time
    first_blood_team
    first_blood_player1
    first_blood_player2
    radiant_bottle_time
    radiant_courier_time
    radiant_flying_courier_time
    radiant_first_ward_time
    dire_bottle_time
    dire_courier_time
    dire_flying_courier_time
    dire_first_ward_time
   
    Я эту вашу Доту не знаю, но полагаю, что в первые пять минут игры c игроками чаще всего происходят события, описываемые признаками courier и first_ward. Соответственно, наименьшее кол-во пропусков в данных есть именно в этих признаках, поскольку эти пропуски, очевидно, означают отсутствие события, описываемого признаком в первые пять минут игры. Сourier, наверное, какой-то отправленный курьер, first_ward – первое отражение атаки. Далее идёт убывание вероятности события в первые пять минут вплоть до признака first_blood_player2, каковой, наверное, есть игрок, причинивший ущерб first_blood_player1, поскольку этот самый first_blood_player может быть не героем, а ботом, миной или ещё чем.
    
   #### 2. Как называется столбец, содержащий целевую переменную?
    radiant_win
   
   #### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?
    Кросс-валидация для для градиентного бустинга с 30 деревьями длилась 1 минуту 32 секунды (или 1 минута 56 секунд вместе с классификацией и обучением классификатора). Качество при этом равно 0.6892847426135573.
   
   #### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
    Разница в качестве между кросс-валидацией для классификаторов на 10 и 20 деревьях 0.02, на 20 и 30 – 0.007 (в три раза меньше), на 30 и 40 – 0.005 и далее разница в качестве падает до 0.001, что выходит при 90 и 100 деревьях. При этом разница во времени, затрачиваемом на кросс-валидацию растёт обратно пропорционально уменьшению разницы в качестве. Если при 30 деревьях кросс-валидация занимает 1 минуту 32 секунды, то при 100 – 3 минуты 38 секунд. При этом качество повышается лишь на 0.017. Что, скорее всего, несущественно.
    
    Чтобы ускорить обучение классификатора на градиентном бустинге при увеличении кол-ва деревьев можно сокращать количество признаков. В первую очередь, проверить, признаки на корреляцию и отфильтровать те, что коррелируют сильно, также можно снизить размерность, заменив старые признаки на новые, значительно меньшие по количеству, но не снижающие дисперсию.

## Подход 2: логистическая регрессия

In [1]:
import numpy as np
import pandas as pd

features = pd.read_csv('../features.csv', index_col='match_id')
features_test = pd.read_csv('../features_test.csv', index_col='match_id')

X_train = features.drop(['start_time',
                         'lobby_type',
                         'duration',
                         'radiant_win',
                         'tower_status_radiant',
                         'tower_status_dire',
                         'barracks_status_radiant',
                         'barracks_status_dire'], axis=1)
Y_train = features['radiant_win']
X_test = features_test.drop(['start_time', 'lobby_type'], axis=1)

X_train.fillna(0, inplace=True, axis=1)
X_test.fillna(0, inplace=True, axis=1)

Нормализуйем обучающую и тестовую выборку с помощью класса StandardScaler

In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # сначала fit_transform, потом transform
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape, Y_train.shape)

(97230, 100) (97230,)


Подберём наилучшие значение параметра регуляризации

In [16]:
arr = list(map(lambda x: 10**2 / 10**x, range(11)))[::-1]
arr

[1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

In [17]:
from sklearn.linear_model import LogisticRegression as lr
from sklearn.model_selection import KFold, cross_val_score

for c in arr:    # C (regularization)
    clf_lr = lr(C=c).fit(X_train_scaled, Y_train)
    folder_lr = KFold(n_splits=5, shuffle=True)    # генератор разбиений
    print(c, '\t', np.mean(cross_val_score(clf_lr, X_train_scaled, Y_train, scoring='roc_auc', cv=folder_lr)))

1e-08 	 0.6871681046904048
1e-07 	 0.6871827193666747
1e-06 	 0.688094021117126
1e-05 	 0.6950863020674684
0.0001 	 0.7112542442678166
0.001 	 0.7159357523775383
0.01 	 0.716369200090057
0.1 	 0.7162368607354933
1.0 	 0.7161246149394107
10.0 	 0.7162948948118885
100.0 	 0.7162599423193535


Уберём категориальные признаки и снова подберём наилучшие значение параметра регуляризации

In [18]:
X_train_nc = X_train.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                           'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X_train_nc_scaled = scaler.fit_transform(X_train_nc)  # сначала fit_transform, потом transform

for c in arr:    # C (regularization)
    clf_nc_lr = lr(C=c).fit(X_train_nc_scaled, Y_train)
    folder_nc_lr = KFold(n_splits=5, shuffle=True)    # генератор разбиений
    print(c, '\t', np.mean(cross_val_score(clf_nc_lr, X_train_nc_scaled, Y_train, scoring='roc_auc', cv=folder_nc_lr)))

1e-08 	 0.6871760248116618
1e-07 	 0.6871999234035899
1e-06 	 0.6881203430057555
1e-05 	 0.6950646014259245
0.0001 	 0.711222990683126
0.001 	 0.7162803639650029
0.01 	 0.7165203285397959
0.1 	 0.7164235157010062
1.0 	 0.7163020788875458
10.0 	 0.7162645914753971
100.0 	 0.7163362821312302


Всего 112 героев, уложены в массив heroes

In [19]:
heroes = features['r1_hero'].unique()
n = max(heroes)
n

112

Сформируем мешок слов (bag of words)

In [20]:
heroes = np.unique(features['r1_hero'])
n = max(heroes)
X_pick = np.zeros((features.shape[0], n))
for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [21]:
print(sum(abs(X_pick[1000, :])), sum(X_pick[1000, :]))

10.0 0.0


In [22]:
X_train_bow = np.hstack((X_train_nc, X_pick))
print(X_train_nc.shape, X_train_bow.shape)

(97230, 90) (97230, 202)


мешок слов для тестовой выборки

In [23]:
X_test_nc = X_test.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                           'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X_pick_test = np.zeros((features_test.shape[0], n))
for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick_test[i, features_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test_bow = np.hstack((X_test_nc, X_pick_test))

In [24]:
print(X_pick_test.shape, sum(abs(X_pick_test[1000, :])), sum(X_pick_test[1000, :]))

(17177, 112) 10.0 0.0


Кросс-валидация для обучающей выборки с мешком слов

In [25]:
X_train_bow_scaled = scaler.fit_transform(X_train_bow)  # сначала fit_transform, потом transform

for c in arr:    # C (regularization)
    clf_bow_lr = lr(C=c).fit(X_train_bow_scaled, Y_train)
    folder_bow_lr = KFold(n_splits=5, shuffle=True)    # генератор разбиений
    print(c, '\t', np.mean(cross_val_score(clf_bow_lr, X_train_bow_scaled, Y_train, scoring='roc_auc', cv=folder_bow_lr)))

1e-08 	 0.7005702802580507
1e-07 	 0.7007917660136536
1e-06 	 0.7021673976732454
1e-05 	 0.7146779169738959
0.0001 	 0.742463825649414
0.001 	 0.7513976947894491
0.01 	 0.7518204021963442
0.1 	 0.7519297370162686
1.0 	 0.7514110452592099
10.0 	 0.7515952988807804
100.0 	 0.7517566492402846


Получим предсказания вероятностей победы команды Radiant в играх тестовой выборки с логистической регрессией в кач-ве классификатора. Наилучший результат при кросс-валидации на обучающей выборке оказался на параметре регуляризации 0.1, его и применим.

In [26]:
X_test_bow_scaled = scaler.transform(X_test_bow)

prediction = clf_bow_lr.predict_proba(X_test_bow_scaled)[:, 1]

In [30]:
print(prediction, '\n', min(prediction), max(prediction))

[0.83651972 0.77532749 0.20130384 ... 0.22950449 0.61973228 0.41646966] 
 0.008376838227443784 0.9966080664850568


## Отчёт по второму этапу:

   #### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
    Признаки были отмасштабированы. Кроме признаков из будущего, были убраны start_time и lobby_type. Я их счёл не информативными для задачи предсказания результатов игр. Если относительно start_time у меня нет никаких сомнений, то по поводу lobby_type хотелось бы услышать мнение тех, кто играл а эту вашу Доту: а вдруг да тип лобби где собираются игроки влияет на результат игры.
    Качество получилось близким, 0.716369200090057 на параметре регуляризации 0.01. Сходно, чуть выше, чем при градиентном бустинге.
    Логистическая регрессия работает заметно (кратно) быстрее градиентного бустинга.
    
   #### Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
    Удаление категориальных признаков улучшает качество логистической регрессии. Хотя и не особо заметно, в моём случае, меньше 1%
     
   #### Сколько различных идентификаторов героев существует в данной игре?
    112
   #### Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
    При добавлении "мешка слов" по героям качество получилось на 5% лучше (0.7519)
   #### Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
    0.008376838227443784 и 0.9966080664850568 соответственно.